# Embeddings Multilingües: Semántica más allá del Idioma
## Introducción
En este demo, exploraremos cómo los modelos de lenguaje modernos pueden entender el **significado** de un texto independientemente del idioma en el que esté escrito.

Utilizaremos:
1.  **Sentence Transformers**: Para convertir texto en vectores numéricos (embeddings).
2.  **t-SNE**: Para reducir la dimensionalidad y visualizar los vectores en 3D.
3.  **Plotly**: Para gráficos interactivos.

El objetivo es ver cómo reseñas que hablan de lo mismo (e.g., "batería") se agrupan juntas, aunque una esté en inglés y otra en español.

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
import plotly.express as px

# Configuración para mostrar texto completo en pandas
pd.set_option('display.max_colwidth', None)

## 1. Carga y Etiquetado de Datos
Cargamos las reseñas de Amazon. Como son pocas, añadiremos manualmente etiquetas de **Idioma** e **Idea Principal** para facilitar la visualización.

In [2]:
# Cargar datos originales
df = pd.read_csv('../../data/resenas_amazon.csv')

# Añadir etiquetas manuales (basado en análisis previo)
# 0: Inglés - Respaldo/Batería
# 1: Español - Respaldo/Batería
# 2: Inglés - Fallo/Defectuoso
# 3: Portugués - Protección
# 4: Portugués - Protección
# 5: Japonés - General/Monitorización
# 6: Japonés - General/Buen producto

etiquetas = [
    {'Idioma': 'Inglés', 'Idea': 'Respaldo/Batería'},
    {'Idioma': 'Español', 'Idea': 'Respaldo/Batería'},
    {'Idioma': 'Inglés', 'Idea': 'Fallo/Defectuoso'},
    {'Idioma': 'Portugués', 'Idea': 'Protección'},
    {'Idioma': 'Portugués', 'Idea': 'Protección'},
    {'Idioma': 'Japonés', 'Idea': 'General'},
    {'Idioma': 'Japonés', 'Idea': 'General'}
]

# Unir al DataFrame
df_etiquetas = pd.DataFrame(etiquetas)
df = pd.concat([df, df_etiquetas], axis=1)

# Mostrar datos
df[['Idioma', 'Idea', 'Reseña']]

,Idioma,Idea,Reseña
0,Inglés,Respaldo/Batería,"The CyberPower CP1500PFCLCD has been an absolute lifesaver during major power outages! This UPS consistently provides over 90 minutes of backup power for my entire setup—two 32"" monitors, a laptop, and my modem—allowing me to work seamlessly even when the power is out. I was amazed at how well it handles the load , keeping everything running smoothly\nThe LCD screen is super handy, providing real-time information on battery capacity and estimated runtime, which gives me peace of mind knowing exactly how much backup time I have left. It’s also well-built and compact enough to fit under my desk without taking up too much space.\n\nFor anyone needing a reliable UPS that can handle multiple devices during outages, the CyberPower CP1500PFCLCD is a fantastic investment. It’s perfect for home or office use, atoo I highly recommend it to anyone who can’t afford to lose power onupitical devices. This UPS is worth every dollar!"
1,Español,Respaldo/Batería,"Es la onda tener respaldo, al menos para los router WiFi doméstico, tengo dos conectados y les da una respaldo de 170min, obvio si conectas algo de mayor consumo tu respaldo será menor. Tiene bastantes contactos solo una fila esta a la batería la otra es normal."
2,Inglés,Fallo/Defectuoso,"The product worked for awhile. It prevented some surges.\n\nUnfortunately, the battery is either garbage or it was defective in my case. It died in 1.5 years of normal use.\n\nWhen I did a ""self-test"" it literally cut the power to everything and printed an error code ""E21"".\n\nI never used the battery itself, except in one short power outage."
3,Portugués,Protección,"Ja impediu meu pc de desligar quando a luz caiu , mas teve algumas poucas vezes que falhou em fazer isso . Não tem muito o que dizer , mas depois de perder peças caras do meu pc por um curto , eu recomendaria ter um desses , mesmo se não esse modelo , apenas tenha certeza de ser um nobreak , e não um estabilizador , nobreak te salva , estabilizador só te ajuda ."
4,Portugués,Protección,"Um equipamento que atende para proteger sua TV, vídeo game e outros periféricos de baixo consumo. E apesar de ser vendido como usado, está com aparência de novo e funcionando. Recomendo."
5,Japonés,General,まず、このUPSに全体的に非常に満足し、感銘を受けていると言うことから始めます。2つの理由で5つ星ではなく4つ星と評価しました。一番の理由は、PC用に1500lcdを購入し、とても気に入ったことです。この850をモデム、ルーター、セキュリティカメラNvrのためだけに購入しました。その後、家の両端にあるセキュリティカム専用に、小さな850valcdユニットを2つ購入しました。驚いたことに、他のユニットが届いたとき、USBケーブルですべてを監視しました私のパソコンから...WA WA 笑いいえ、高額な料金を支払わない限り、監視できるのは1つだけで、ユニットを切り替える方法はありません。スイッチ付きのUSBハブにさらに100ドルを費やしましたが、それでも問題なく動作しますが、プログラムを閉じて、USBを切り替えて、アップごとに再度開く必要があります。私にはまったくかっこよくありません笑購入したら1つ購入すると書かれています。2つ目の問題は、バッテリーの状態を監視できるという広告です。2つ目の問題は、バッテリーの状態を監視できると書いてありますが、実際には、月額80ドルのクラウドプログラムに登録していない限り監視できません。全体的には良い製品ですが、よくわかりません会社自体。多くの人が「すべてをカバーするUPSを1つ買う」と言っています。私の場合、それらは数百フィート離れているので、1台はそうではありません仕事。一度に複数のアップを監視できないなんて夢にも思わなかった。だから、ソフトウェアの改善で評価を5に変更するよ。\nレポート\nAmazonによって英語から翻訳されました
6,Japonés,General,素晴らしい製品、セットアップが簡単（プラグを差し込むだけ）、素敵なディスプレイ画面、便利なサイズ。\nレポート


## 2. Generación de Embeddings
Usaremos el modelo `paraphrase-multilingual-MiniLM-L12-v2`. Este modelo ha sido entrenado para entender más de 50 idiomas y mapear oraciones con significados similares a vectores cercanos en el espacio vectorial.

In [3]:
# Cargar modelo multilingüe
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Generar embeddings
# Esto convierte cada reseña en un vector de 384 números
embeddings = model.encode(df['Reseña'].tolist())

print(f"Dimensiones de los embeddings: {embeddings.shape}")
print("Ejemplo de los primeros 5 valores del primer vector:")
print(embeddings[0][:5])

Dimensiones de los embeddings: (7, 384)
Ejemplo de los primeros 5 valores del primer vector:
[-0.12546927  0.10557793 -0.1401968  -0.09745549  0.20322707]


## 3. Reducción de Dimensionalidad (t-SNE)
No podemos visualizar 384 dimensiones. Usaremos **t-SNE** (t-Distributed Stochastic Neighbor Embedding) para reducir estos vectores a solo 3 dimensiones, preservando la cercanía relativa entre ellos.

In [4]:
# Configurar t-SNE para 3 componentes (3D)
# perplexity bajo porque tenemos muy pocos datos (7 muestras)
tsne = TSNE(n_components=3, perplexity=2, random_state=42, init='pca', learning_rate=200)

projections = tsne.fit_transform(embeddings)

# Añadir las proyecciones al DataFrame para graficar
df['x'] = projections[:, 0]
df['y'] = projections[:, 1]
df['z'] = projections[:, 2]

print("Proyecciones 3D generadas.")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Proyecciones 3D generadas.


## 4. Visualización Interactiva 3D
Ahora graficaremos los puntos en 3D.
*   **Color**: Representa la **Idea Principal**.
*   **Símbolo**: Representa el **Idioma**.

**¡Explora el gráfico!** Gira, acerca y pasa el mouse sobre los puntos para ver la reseña. Deberías ver que los puntos del mismo color (misma idea) están cerca, sin importar el símbolo (idioma).

In [5]:
fig = px.scatter_3d(
    df, x='x', y='y', z='z',
    color='Idea', symbol='Idioma',
    hover_data=['Reseña'],
    title='Embeddings Multilingües: Agrupamiento Semántico',
    opacity=0.9,
    size_max=20
)

# Ajustar tamaño de los puntos
fig.update_traces(marker=dict(size=12))

fig.show()